<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
# use requests.get() method with the provided static_url
# assign the response to a object

import requests

# URL of the Falcon9 Launch Wiki page
static_url = 'https://en.wikipedia.org/wiki/Falcon_9'  # Replace with the correct URL if needed

# Perform the HTTP GET request and store the response
response = requests.get(static_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    print("Request successful")
else:
    print(f"Request failed with status code: {response.status_code}")


Request successful


Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content

import requests
from bs4 import BeautifulSoup

# URL of the Falcon9 Launch Wiki page
static_url = 'https://en.wikipedia.org/wiki/Falcon_9'  # Replace with the correct URL if needed

# Perform the HTTP GET request and store the response
response = requests.get(static_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    print("Request successful")
    
    # Create a BeautifulSoup object to parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Now you can work with the 'soup' object to extract data from the HTML page
    # For example, print the title of the page
    print(soup.title.text)
else:
    print(f"Request failed with status code: {response.status_code}")


Request successful
Falcon 9 - Wikipedia


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [7]:
# Use soup.title attribute
import requests
from bs4 import BeautifulSoup

# URL of the Falcon9 Launch Wiki page
static_url = 'https://en.wikipedia.org/wiki/Falcon_9'  # Replace with the correct URL if needed

# Perform the HTTP GET request and store the response
response = requests.get(static_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    print("Request successful")
    
    # Create a BeautifulSoup object to parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Print the page title
    print(soup.title.text)  # This will print the title of the page
else:
    print(f"Request failed with status code: {response.status_code}")


Request successful
Falcon 9 - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [8]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`

import requests
from bs4 import BeautifulSoup

# URL of the Falcon9 Launch Wiki page
static_url = 'https://en.wikipedia.org/wiki/Falcon_9'  # Replace with the correct URL if needed

# Perform the HTTP GET request and store the response
response = requests.get(static_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    print("Request successful")
    
    # Create a BeautifulSoup object to parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all tables on the page
    html_tables = soup.find_all('table')
    
    # Print the number of tables found and the first table to check
    print(f"Found {len(html_tables)} tables.")
    print(html_tables[0])  # Displaying the first table to check
else:
    print(f"Request failed with status code: {response.status_code}")


Request successful
Found 25 tables.
<table class="infobox hproduct"><caption class="infobox-title">Falcon 9</caption><tbody><tr><td class="infobox-image" colspan="2"><span class="mw-default-size" typeof="mw:File/Frameless"><a class="mw-file-description" href="/wiki/File:Falcon_9_logo.svg"><img alt="Logo of the Falcon 9" class="mw-file-element" data-file-height="162" data-file-width="230" decoding="async" height="49" src="//upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Falcon_9_logo.svg/70px-Falcon_9_logo.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Falcon_9_logo.svg/105px-Falcon_9_logo.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Falcon_9_logo.svg/140px-Falcon_9_logo.svg.png 2x" width="70"/></a></span></td></tr><tr><td class="infobox-image" colspan="2"><span class="mw-default-size" typeof="mw:File/Frameless"><a class="mw-file-description" href="/wiki/File:SpaceX_Demo-2_Launch_(NHQ202005300044)_(cropped).jpg"><img alt="Ground-level view o

Starting from the third table is our target table contains the actual launch records.


In [9]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable">
<tbody><tr>
<td>Height
</td>
<td>41.2 m / 135.2 ft
</td></tr>
<tr>
<td>Height (with interstage)
</td>
<td>47.7 m / 156.5 ft
</td></tr>
<tr>
<td>Diameter
</td>
<td>3.7 m / 12 ft
</td></tr>
<tr>
<td>Empty Mass
</td>
<td>25,600 kg / 56,423 lb
</td></tr>
<tr>
<td>Propellant Mass
</td>
<td>395,700 kg/ 872,369 lb
</td></tr>
<tr>
<td>Structure Type
</td>
<td>LOX tank: <a href="/wiki/Monocoque" title="Monocoque">monocoque</a>
</td></tr>
<tr>
<td>
</td>
<td>Fuel tank: skin and stringer
</td></tr>
<tr>
<td>Structure Material
</td>
<td>Aluminum lithium skin; aluminum domes
</td></tr>
<tr>
<td>Landing Legs
</td>
<td>Number: 4
</td></tr>
<tr>
<td>
</td>
<td>Material: <a class="mw-redirect" href="/wiki/Carbon_fiber" title="Carbon fiber">carbon fiber</a>; <a class="mw-redirect" href="/wiki/Aluminum_honeycomb" title="Aluminum honeycomb">aluminum honeycomb</a>
</td></tr>
<tr>
<td>Number of Merlin Engines
</td>
<td>9 sea level
</td></tr>
<tr>
<td>Propellant
</td>
<td>LOX / RP-1

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [11]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names

from bs4 import BeautifulSoup

# Assuming 'first_launch_table' is already extracted as per the previous task
# Now, let's extract column names from the <th> elements in the header row

# Extract all the <th> elements from the table header
header_columns = first_launch_table.find_all('th')

# Define a function to extract the cleaned column name
def extract_column_from_header(th_element):
    if th_element:
        # Clean up the column name by extracting the text and stripping any surrounding whitespace
        return th_element.get_text(strip=True)
    return None

# List to store the column names
column_names = []

# Iterate through each <th> element and extract the column names
for th in header_columns:
    column_name = extract_column_from_header(th)
    if column_name and len(column_name) > 0:
        column_names.append(column_name)

# Print the extracted column names
print(column_names)



[]


Check the extracted column names


In [17]:
# Print the first launch table to inspect its content
print(first_launch_table)

# Extract all the <th> elements from the table header
header_columns = first_launch_table.find_all('th')

# Check what the header columns look like
print(f"Number of <th> elements found: {len(header_columns)}")
for th in header_columns:
    print(th)

# Define a function to extract the cleaned column name
def extract_column_from_header(th_element):
    if th_element:
        # Clean up the column name by extracting the text and stripping any surrounding whitespace
        return th_element.get_text(strip=True)
    return None

# List to store the column names
column_names = []

# Iterate through each <th> element and extract the column names
for th in header_columns:
    column_name = extract_column_from_header(th)
    if column_name and len(column_name) > 0:
        column_names.append(column_name)

# Print the extracted column names
print("Extracted Column Names:", column_names)


<table class="wikitable">
<tbody><tr>
<td>Height
</td>
<td>41.2 m / 135.2 ft
</td></tr>
<tr>
<td>Height (with interstage)
</td>
<td>47.7 m / 156.5 ft
</td></tr>
<tr>
<td>Diameter
</td>
<td>3.7 m / 12 ft
</td></tr>
<tr>
<td>Empty Mass
</td>
<td>25,600 kg / 56,423 lb
</td></tr>
<tr>
<td>Propellant Mass
</td>
<td>395,700 kg/ 872,369 lb
</td></tr>
<tr>
<td>Structure Type
</td>
<td>LOX tank: <a href="/wiki/Monocoque" title="Monocoque">monocoque</a>
</td></tr>
<tr>
<td>
</td>
<td>Fuel tank: skin and stringer
</td></tr>
<tr>
<td>Structure Material
</td>
<td>Aluminum lithium skin; aluminum domes
</td></tr>
<tr>
<td>Landing Legs
</td>
<td>Number: 4
</td></tr>
<tr>
<td>
</td>
<td>Material: <a class="mw-redirect" href="/wiki/Carbon_fiber" title="Carbon fiber">carbon fiber</a>; <a class="mw-redirect" href="/wiki/Aluminum_honeycomb" title="Aluminum honeycomb">aluminum honeycomb</a>
</td></tr>
<tr>
<td>Number of Merlin Engines
</td>
<td>9 sea level
</td></tr>
<tr>
<td>Propellant
</td>
<td>LOX / RP-1

## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [ ]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [21]:
import pandas as pd

# Initialize an empty dictionary with keys as column names
launch_dict = dict.fromkeys(column_names)

# Check if the key 'Date and time ( )' exists before deleting it
if 'Date and time ( )' in launch_dict:
    del launch_dict['Date and time ( )']

# Initialize the dictionary with empty lists
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

# Function to parse date and time from the date_time column
def date_time(date_str):
    # Splitting the date and time
    try:
        date_list = date_str.split(' ')  # Example: "Jan 1, 2020 12:00"
        return [date_list[0], date_list[1]]  # [Date, Time]
    except Exception as e:
        return ["N/A", "N/A"]

# Function to extract booster version
def booster_version(booster_str):
    # Extract booster version if present
    if booster_str and 'booster' in booster_str.lower():
        return booster_str.get_text(strip=True)
    return None

# Function to extract payload mass (this is just an example)
def get_mass(mass_str):
    # Example logic for extracting mass
    if mass_str:
        mass = mass_str.get_text(strip=True)
        return mass if mass != 'N/A' else None
    return None

# Function to extract landing status of the booster
def landing_status(landing_str):
    # Example logic to determine landing status
    if landing_str and 'landed' in landing_str.lower():
        return 'Landed'
    elif landing_str and 'failed' in landing_str.lower():
        return 'Failed'
    else:
        return 'Unknown'

# Extract each table
extracted_row = 0
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Iterate through table rows
    for rows in table.find_all("tr"):
        # Check if the row has a header cell with a flight number
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        
        # Get table elements (cells) from the row
        row = rows.find_all('td')
        
        # If it's a valid row (flag=True), extract data and append to the dictionary
        if flag:
            extracted_row += 1

            # Flight Number value
            launch_dict['Flight No.'].append(flight_number)

            # Date and Time
            datatimelist = date_time(row[0].text)
            launch_dict['Date'].append(datatimelist[0].strip(','))
            launch_dict['Time'].append(datatimelist[1])

            # Booster Version
            bv = booster_version(row[1])
            if not bv:
                bv = row[1].a.string if row[1].a else None
            launch_dict['Version Booster'].append(bv)

            # Launch Site
            launch_site = row[2].a.string if row[2].a else None
            launch_dict['Launch site'].append(launch_site)

            # Payload
            payload = row[3].a.string if row[3].a else None
            launch_dict['Payload'].append(payload)

            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)

            # Orbit
            orbit = row[5].a.string if row[5].a else None
            launch_dict['Orbit'].append(orbit)

            # Customer
            customer = row[6].a.string if row[6].a else None
            launch_dict['Customer'].append(customer)

            # Launch Outcome
            launch_outcome = list(row[7].strings)[0] if row[7] else 'Unknown'
            launch_dict['Launch outcome'].append(launch_outcome)

            # Booster Landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

# Convert the dictionary into a DataFrame
launch_df = pd.DataFrame(launch_dict)

# Display the DataFrame
launch_df.head()


,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [23]:
# Create DataFrame from launch_dict while explicitly specifying the dtype for Series
df = pd.DataFrame({key: pd.Series(value, dtype="object") for key, value in launch_dict.items()})

# Display the DataFrame
df.head()


,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
